In [43]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers

import h5py

In [3]:
# images dimension
img_width, img_height = 150, 150

train_data_dir = 'dataset/train'
test_data_dir = 'dataset/test'

In [4]:
# used to rescale the pixel values from [0, 255] to [0, 1] interval

datagen = ImageDataGenerator(rescale=1./255)

In [5]:
# automagically retrieve images and their classes for train and validation sets

train_generator = datagen.flow_from_directory(
                    train_data_dir,
                    target_size=(img_width, img_height), 
                    batch_size=16, class_mode='binary')

test_generator = datagen.flow_from_directory(
                    test_data_dir,
                    target_size=(img_width, img_height),
                    batch_size=32, class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 802 images belonging to 2 classes.


## Small Conv Net

### Model architecture definition

In [6]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [7]:
model.compile(loss='binary_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])

## Training

In [8]:
nb_epoch = 30
nb_train_samples = 2000
nb_test_samples = 802

In [20]:
model.fit(train_generator, 
          epochs=nb_epoch, 
          validation_data=test_generator)

Epoch 1/30
125/125 [==============================] - 7s 57ms/step - loss: 0.6615 - accuracy: 0.6000 - val_loss: 0.6473 - val_accuracy: 0.5885
Epoch 2/30
125/125 [==============================] - 7s 58ms/step - loss: 0.6315 - accuracy: 0.6535 - val_loss: 0.6230 - val_accuracy: 0.6384
Epoch 3/30
125/125 [==============================] - 8s 64ms/step - loss: 0.6026 - accuracy: 0.6740 - val_loss: 0.5979 - val_accuracy: 0.6820
Epoch 4/30
125/125 [==============================] - 7s 56ms/step - loss: 0.5784 - accuracy: 0.7005 - val_loss: 0.6223 - val_accuracy: 0.6496
Epoch 5/30
125/125 [==============================] - 7s 57ms/step - loss: 0.5518 - accuracy: 0.7325 - val_loss: 0.5790 - val_accuracy: 0.7082
Epoch 6/30
125/125 [==============================] - 7s 56ms/step - loss: 0.5373 - accuracy: 0.7370 - val_loss: 0.5815 - val_accuracy: 0.6895
Epoch 7/30
125/125 [==============================] - 7s 55ms/step - loss: 0.5050 - accuracy: 0.7515 - val_loss: 0.5778 - val_accuracy: 0.6970

In [29]:
# Evaluating on validation set 

model.evaluate_generator(test_generator, nb_test_samples)

[1.1790086030960083, 0.7219451665878296]

## Data augmentation for improving the model¶

In [31]:
train_datagen_augmented = ImageDataGenerator(
    rescale=1./255,       # normalize pixel values to [0,1]
    shear_range=0.2,      # randomly applies shearing transformation
    zoom_range=0.2,       # randomly applies shearing transformation
    horizontal_flip=True) # randomly flip the images

In [32]:
train_generator_augmented = train_datagen_augmented.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary')

Found 2000 images belonging to 2 classes.


In [40]:
model.fit_generator(
    train_generator_augmented,
    steps_per_epoch=2000,
    epochs=30,
    validation_data=test_generator,
    
)

Epoch 1/30
2000/2000 [==============================] - 18s 9ms/step - loss: 0.5353 - accuracy: 0.7365 - val_loss: 0.5652 - val_accuracy: 0.7195


In [41]:
model.evaluate_generator(test_generator, nb_test_samples)

/usr/local/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.565195620059967, 0.719451367855072]

## VGG16 + small MLP

In [75]:
model_vgg = Sequential()
model_vgg.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height, 3)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model_vgg.add(MaxPooling2D((1, 1), strides=(2, 2), padding='valid'))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model_vgg.add(MaxPooling2D((1, 1), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model_vgg.add(MaxPooling2D((1, 1), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model_vgg.add(MaxPooling2D((1, 1), strides=(2, 2)))

### Using the VGG16 model to process samples

In [69]:
train_generator_bottleneck = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode=None,
    shuffle=False)

test_generator_bottleneck = datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode=None,
    shuffle=False)

Found 2000 images belonging to 2 classes.
Found 802 images belonging to 2 classes.


In [79]:
train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))
test_labels = np.array([0] * (nb_test_samples // 2) + [1] * (nb_test_samples // 2))

In [91]:
bottleneck_features_train = model_vgg.predict_generator(train_generator_bottleneck, nb_train_samples)
bottleneck_features_validation = model_vgg.predict_generator(test_generator_bottleneck, nb_test_samples)

train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))
test_labels = np.array([0] * (nb_test_samples // 2) + [1] * (nb_test_samples // 2))

model_top = Sequential()
model_top.add(Flatten())
model_top.add(Dense(256, activation='relu'))
model_top.add(Dropout(0.5))
model_top.add(Dense(1, activation='sigmoid'))

model_top.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [96]:
model_top.fit(train_generator,
          epochs=40, batch_size=32,
          validation_data=test_generator)

Epoch 1/40
125/125 [==============================] - 14s 109ms/step - loss: 0.7018 - accuracy: 0.4885 - val_loss: 0.6930 - val_accuracy: 0.5037
Epoch 2/40
125/125 [==============================] - 14s 111ms/step - loss: 0.7025 - accuracy: 0.4970 - val_loss: 0.6931 - val_accuracy: 0.4988
Epoch 3/40
125/125 [==============================] - 14s 111ms/step - loss: 0.7198 - accuracy: 0.4880 - val_loss: 0.6930 - val_accuracy: 0.5037
Epoch 4/40
125/125 [==============================] - 14s 114ms/step - loss: 0.7024 - accuracy: 0.4985 - val_loss: 0.7012 - val_accuracy: 0.4863
Epoch 5/40
125/125 [==============================] - 14s 111ms/step - loss: 0.6979 - accuracy: 0.4870 - val_loss: 0.6930 - val_accuracy: 0.5025
Epoch 6/40
125/125 [==============================] - 14s 112ms/step - loss: 0.7294 - accuracy: 0.4875 - val_loss: 0.6924 - val_accuracy: 0.5025
Epoch 7/40
125/125 [==============================] - 14s 112ms/step - loss: 0.6955 - accuracy: 0.4945 - val_loss: 0.6903 - val_ac

In [99]:
model_top.evaluate_generator(test_generator, nb_test_samples)

[0.6699514985084534, 0.5448877811431885]

In [100]:
model_vgg = Sequential()
model_vgg.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height, 3)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model_vgg.add(MaxPooling2D((1, 1), strides=(2, 2), padding='valid'))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model_vgg.add(MaxPooling2D((1, 1), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model_vgg.add(MaxPooling2D((1, 1), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model_vgg.add(MaxPooling2D((1, 1), strides=(2, 2)))

In [101]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model_vgg.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

In [102]:
for layer in model_vgg.layers[:25]:
    layer.trainable = False

In [103]:
model_vgg.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [105]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 802 images belonging to 2 classes.


In [ ]:
model_vgg.fit(train_generator,
          epochs=10,
          validation_data=test_generator)

Epoch 1/10
63/63 [==============================] - 20s 314ms/step - loss: 5.0224 - accuracy: 0.0000e+00 - val_loss: 5.0169 - val_accuracy: 0.0000e+00
Epoch 2/10
63/63 [==============================] - 20s 314ms/step - loss: 5.0122 - accuracy: 0.0000e+00 - val_loss: 5.0075 - val_accuracy: 0.0000e+00
Epoch 3/10
63/63 [==============================] - 20s 323ms/step - loss: 5.0036 - accuracy: 0.0000e+00 - val_loss: 4.9998 - val_accuracy: 0.0000e+00
Epoch 4/10
63/63 [==============================] - 21s 327ms/step - loss: 4.9962 - accuracy: 0.0000e+00 - val_loss: 4.9927 - val_accuracy: 0.0000e+00
Epoch 5/10
63/63 [==============================] - 21s 326ms/step - loss: 4.9897 - accuracy: 0.0000e+00 - val_loss: 4.9867 - val_accuracy: 0.0000e+00
Epoch 6/10
63/63 [==============================] - 21s 325ms/step - loss: 4.9840 - accuracy: 0.0000e+00 - val_loss: 4.9814 - val_accuracy: 0.0000e+00
Epoch 7/10
63/63 [==============================] - 21s 325ms/step - loss: 4.9790 - accuracy: 

In [108]:
model_vgg.evaluate_generator(test_generator, nb_test_samples)

[5.02806282043457, 0.0]

In [109]:
model.evaluate_generator(test_generator, nb_test_samples)

[0.565195620059967, 0.719451367855072]

In [113]:
model_top.evaluate_generator(test_generator, nb_test_samples)

[0.6699513792991638, 0.5448877811431885]